In [2]:
import sys
sys.path.append("..")

import os
cwd = os.getcwd()

import torch
import numpy as np

from GENUINE.data.datasets import PatchDataset
from torch.utils.data import DataLoader
from GENUINE.utils.device import best_gpu
from GENUINE.utils.visualize import gridPlot
from GENUINE.data.custom_transforms import ToTensor

DATASET_PATH="DATASETS/TEST_DATASET_SMALL.h5"
MODELS_PATH="TRAINED_MODELS"

/home/simon_g/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = PatchDataset(os.path.join(cwd, DATASET_PATH), dataset="TEST", transform=[ToTensor()])
dl = DataLoader(ds, batch_size=8)

device = best_gpu(verbose=False)

for m in os.listdir(os.path.join(cwd, MODELS_PATH)):
    
    if "RetinaNet" in m:
        continue
    
    model = torch.load(os.path.join(cwd, f"{MODELS_PATH}/{m}"), map_location="cpu")
    model.eval()
    
    print(model.__class__)
    
    model.to(device)
    
    preds, patches, targets = [], [], []
    for X, y in dl:
        
        preds.extend((model(X.to(device)) > 0).int().squeeze().tolist())
        patches.append(X.numpy().transpose(0, 2, 3, 1))
        targets.extend(y.tolist())
        
    gridPlot(np.vstack(patches), labels=preds, targets=targets)

<class 'GENUINE.ModelZoo.GENUINE.GENUINE'>


KeyboardInterrupt: 